# Random Data Simulation and Fitting Using Neural Network

### _Simulation of random data:_

Import necessary packages:

In [1]:
from metabolabpytools import isotopomerAnalysis
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Function to generate random percentages summing to 100%:

In [4]:
def generate_random_percentages(num_isotopomers):
    random_values = np.random.rand(num_isotopomers)
    random_percentages = (random_values / random_values.sum()) * 30  # sum to 30
    return list(random_percentages)

Create an isotopomerAnalysis object:

In [7]:
ia = isotopomerAnalysis.IsotopomerAnalysis()

Define metabolite parameters:

In [9]:
isotopomers = [[0,1,1], [1,0,1], [0,0,1]]
random_percentages = generate_random_percentages(len(isotopomers))
print(random_percentages)
hsqc = [0,1,1]
metabolite = 'L-LacticAcid'
exp_index = 0

[11.06730360033677, 7.0269903305145975, 11.905706069148632]


Initialise isotopomerAnalysis object with selected data:

In [12]:
ia.init_metabolite(metabolite, hsqc)
ia.set_fit_isotopomers(metabolite=metabolite, isotopomers=isotopomers, percentages=random_percentages)

In [14]:
print(f'Isotopomers : {ia.fit_isotopomers[metabolite]}\nIsotopomer %: {ia.isotopomer_percentages[metabolite]}')

Isotopomers : [[0, 0, 0], [0, 1, 1], [1, 0, 1], [0, 0, 1]]
Isotopomer %: [70.0, 11.06730360033677, 7.0269903305145975, 11.905706069148632]


Simulate HSQC data:

In [17]:
ia.sim_hsqc_data(metabolite=metabolite, exp_index=exp_index, isotopomers=isotopomers, percentages=random_percentages)

In [19]:
print(f'HSQC Multiplets: {ia.exp_multiplets[metabolite][exp_index]}')
print(f'Multiplet percentages: {ia.exp_multiplet_percentages[metabolite][exp_index]}')

HSQC Multiplets: [[2], [2, 3], [2, 1, 3], [3], [3, 2]]
Multiplet percentages: [6.231860080871374, 92.15726694713968, 1.61087297198894, 63.34878060485491, 36.65121939514509]


Simulate GC-MS data:

In [30]:
ia.sim_gcms_data(metabolite, exp_index)

In [32]:
ia.exp_gcms[metabolite][exp_index]

[70.0, 11.905706069148632, 18.094293930851368, 0.0]

### _Fitting to simulated data using basic neural network:_

Use only HSQC multiplets and GC-MS data for analysis:

In [38]:
ia.use_hsqc_multiplet_data = True
ia.use_gcms_data = True
ia.use_nmr1d_data = False

Generate multiple samples:

In [ ]:
# num_samples = 10  # Example number of samples
# samples = ia.generate_simulated_samples(num_samples)

Call the new neural network fit method:

In [40]:
ia.fit_data_nn(exp_index=exp_index, metabolite=metabolite, fit_isotopomers=isotopomers)



Epoch 1/50

1/1 [==============================] - 1s 960ms/step - loss: 1348.1771
Epoch 2/50
1/1 [==============================] - 0s 12ms/step - loss: 1268.5940
Epoch 3/50
1/1 [==============================] - 0s 2ms/step - loss: 1192.3629
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 1119.2207
Epoch 5/50
1/1 [==============================] - 0s 3ms/step - loss: 1050.6545
Epoch 6/50
1/1 [==============================] - 0s 8ms/step - loss: 987.0451
Epoch 7/50
1/1 [==============================] - 0s 9ms/step - loss: 925.8461
Epoch 8/50
1/1 [==============================] - 0s 0s/step - loss: 867.7558
Epoch 9/50
1/1 [==============================] - 0s 9ms/step - loss: 814.6113
Epoch 10/50
1/1 [==============================] - 0s 5ms/step - loss: 763.5065
Epoch 11/50
1/1 [==============================] - 0s 10ms/step - loss: 714.9641
Epoch 12/50
1/1 [==============================] - 0s 3ms/step - loss: 670.8259
Epoch 13/50
1/1 [=====================

In [42]:
print(f'Fitted Isotoponers : {ia.fitted_isotopomers[metabolite][exp_index]}\nFitted Isotopomer %: {ia.fitted_isotopomer_percentages[metabolite][exp_index]}')  

Fitted Isotoponers : [[0, 0, 0], [0, 1, 1], [1, 0, 1], [0, 0, 1]]
Fitted Isotopomer %: [70.0, 11.06730360033677, 7.0269903305145975, 11.905706069148632]


Export data and analysis to Excel spreadsheet:

In [ ]:
ia.export_data('fittedIsotopomerAnalysis.xlsx')

## Addressing Overfitting: 

To prevent overfitting in my neural network model for predicting isotopomer distributions, several strategies will be implemented:

- First, I'll use a validation set to monitor the model's performance during training, ensuring it maintains its ability to generalize to unseen data. This involves splitting the data into training and validation sets and using early stopping to halt training when the validation loss stops improving, which helps avoid overfitting by preventing the model from learning noise in the training data. 
 
- Additionally, I'll employ dropout layers within the neural network architecture. Dropout randomly deactivates a fraction of neurons during each training step, which forces the network to learn more robust features and reduces reliance on any specific neurons. 

- Regularization techniques, such as L2 regularization, will be used to penalize large weights, discouraging the model from becoming too complex. 

- Finally, I will ensure that the model is trained with an adequate amount of data and, if necessary, use data augmentation techniques to artificially increase the diversity of the training dataset, enhancing the model's ability to generalize.